# FFT analysis on edge node

The below performs FFT on 4 channels of 8K samples<br/> Data is gatherred from the edge noed, as 4 blocks of 16KBytes <br/> For data analysis python scientific computing package **numpy** is used <br/> Data visualization is done with **Bokeh** a framework for interactive datavisualization. <br/>A bokeh server session is hosted, and requires a bokeh server instance to be started, before executing this script. Do this in a terminal with **bokeh serve**

In [ ]:
import socket,struct,time
import numpy as np
import select

from bokeh.client.session import ClientSession
from bokeh.client import push_session
from bokeh.plotting import figure, curdoc
from bokeh.io import show, vform
from bokeh.models.widgets import RadioGroup, RadioButtonGroup

###### Sampling definitions ######
# No of samples per channel
N  = 8192
# sample rate
FS = 20000
# Time between each block can be retrieved, in [mS]
T_COMM = (N*1000)/FS
# 4 sample channels + info data
MSGLEN = 65542  #((8*8192) +4)

###### Comm settings ######
SERVER_ADDRESS = "192.168.1.102"
SERVER_PORT = 7

###### Vibration analyzer / logger class ######
class VibrationLogger(object):
    '''
    classdocs
    '''
   # constuctor
    def __init__(self ):
        # 4 numpy arrays to hold data        
        self.vibData2 = np.arange(N)
                                  
        #[4000][][][]                          
        self.vibData = np.array([[np.arange(N)],[np.arange(N)],[np.arange(N)],[np.arange(N)]])
        self.fftData = np.array([[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)]])
        self.fftData1 = np.array([[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)]])
        self.fftData2 = np.array([[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)]])
        self.fftData3 = np.array([[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)]])
        self.fftDataDisp = np.array([[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)]])
        
        self.tmp1 = np.array([[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)]])
        self.tmp2 = np.array([[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)]])
     
        self.currentRpmValue = 3000;
        self.sampleFlags = 0
        self.channelsActive = [1,0,0,0]
        self.channelSelected = 0
        self.doFFT = 1;

        # get Hanning window constants
        self.FFTwindow = np.hanning(N)
        # Create arrays for plotting X and Y axis
        self.x = np.linspace(0, FS, N)
        self.t1 = 0
        self.t2 = 0
        
        # availible TOOLs 
        TOOLS="resize,crosshair,pan,wheel_zoom,box_zoom,reset,tap,box_select,lasso_select,hover"
        self.plot_kw = dict(tools=TOOLS, h_symmetry=False, v_symmetry=False,  outline_line_color='#595959',)
        
        # plot headings etc
        #self.p = figure(webgl=True,title="FFT test", plot_width=1200, plot_height=600, y_axis_type="log", **self.plot_kw)
        #self.p = figure(webgl=True,title="FFT test", plot_width=1200, plot_height=600, **self.plot_kw)
        self.p = figure(title="FFT", plot_height=600, plot_width=1200, y_range=(0, 400), **self.plot_kw)


        self.channelColors = ["red","blue","green","magenta"]
        self.r = self.p.line(x=[], y=[], color=self.channelColors[1], line_width=1)
        self.p.ygrid.minor_grid_line_color = 'navy'
        self.p.ygrid.minor_grid_line_alpha = 0.1
        self.p.xaxis.axis_label = "Frequency [Hz]"
        self.p.yaxis.axis_label = "Vibration [mG]"
       
        # attach "ds" to data feed
        self.ds = self.r.data_source        
        self.rpmMarker = self.p.triangle(x=[], y=1, size=25, color="red", alpha=0.8)
        self.rpmds = self.rpmMarker.data_source
        
        # channel selctor buttons
        self.channelSelectorButtons = RadioGroup(labels=["Channel 1", "Channel 2", "Channel 3", "Channel 4"], active=0)
        self.channelSelectorButtons.on_click(self.chanSelUpdate)
        show(vform(self.channelSelectorButtons))
        
        self.isTimeFFTAnalysis = RadioButtonGroup(labels=["Time", "FFT"], active=1)
        self.isTimeFFTAnalysis.on_click(self.timeFFTUpdate)
        show(vform(self.isTimeFFTAnalysis))

        # open a session to keep our local document in sync with server
        self.session = push_session(curdoc())
        curdoc().add_periodic_callback(self.update, T_COMM) #800

    def connectTcp(self):
        self.tcpConn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        # set no delay to avoid 200 ms delayed ACK from server - TODO verify !!
        self.tcpConn.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
        self.tcpConn.connect((SERVER_ADDRESS, SERVER_PORT))
        self.tcpConn.setblocking(0) #added     
        print ('connect to NODE %s:%d' % (SERVER_ADDRESS, SERVER_PORT))
     
    def timeFFTUpdate(self, new):
        if(new == 0):
            self.doFFT = 0;
        else:
            self.doFFT = 1;
        
    def chanSelUpdate(self, new):
        self.channelSelected = new
        print 'Channel ' + str(new) + ' selected.'
        self.p.title = "Data from channel :" + str(self.channelSelected+1)

    def doFFTs(self):        
        #update delay line
        np.copyto(self.fftData3, self.fftData2)
        np.copyto(self.fftData2, self.fftData1)
        np.copyto(self.fftData1, self.fftData)

        # perform FFT on the windowed samples
        for n in range(4):
            # use the absolute value to get rid of 
            self.fftData[n] =np.abs((np.fft.rfft(self.vibData[n] * self.FFTwindow ) /N ))
            self.fftData[n,0,0]=0 #zero out DC    
            self.fftData[n,0,1]=0 #zero out DC    
            self.fftData[n,0,2]=0 #zero out DC    

            
        # a simple average
        # ADD ONLY TAKES TWO ARGUMENTS
        self.tmp1= np.add(self.fftData3, self.fftData2)
        self.tmp2= np.add(self.fftData1, self.fftData)
        self.fftDataDisp = np.add(self.tmp1, self.tmp2)
        #(self.fftData3+self.fftData2+self.fftData1+self.fftData)/4
  
    def requestVibData(self):       
        # protocol
        message = 'GET_SAMPLES\x00'
        #request samples
        self.tcpConn.sendall(message)
        #print 'requesting data'
    
    def update(self):
        self.requestVibData()
        readers,_,_ = select.select([self.tcpConn],[],[])
        if readers:
            # struct containing 4 channels RPM and Status flags
            s1 = struct.Struct('8192H 8192H 8192H 8192H H H H')
            #rawData = self.recv_size(MSGLEN)
            rawData = self.tcpConn.recv(MSGLEN)
            
            if(len(rawData) == MSGLEN):
                sortedData = np.asarray(s1.unpack(rawData)[:-3])
                self.vibData = sortedData.reshape(4,8192)
                
                sortedData = np.asarray(s1.unpack(rawData)[-3:])
                self.currentRpmValue    = sortedData[0]
                self.sampleFlags        = sortedData[1]
                self.msSinceLastFrame   = sortedData[2]
                # we got data, compute FFT's
                self.doFFTs()
                # verify time between frames
                #self.t1 = self.t2
                #self.t2 = time.clock()
                #print 't1:',self.t1,'t2:',self.t2, 'delta:',self.t2-self.t1 #(self.t2-self.t1)
                print 'RPM x10:',sortedData[0]
                #,'[1]:',sortedData[1],'[]:',sortedData[2],     
                
            self.ds.data["x"] = self.x
            #self.ds.data["y"] = self.fftData[self.channelSelected,0]     #vib data is [4:8K] -fftData
            self.ds.data["y"] = self.fftDataDisp[self.channelSelected,0]     #vib data is [4:8K] -fftData
            

Here we create an instance of the vibration logger, and starts logging

In [ ]:
print 'sample settings: T_COMM',T_COMM,'[ms]'
vibLog = VibrationLogger()
vibLog.connectTcp()
#    vibLog.addRpmLines()
vibLog.session.show() # open the document in a browser       
vibLog.session.loop_until_closed() # run forever
